Copyright (c) Microsoft Corporation.

Licensed under the MIT License.

# Text2SQL with Semantic Kernel & Azure OpenAI

This notebook demonstrates how the SQL plugin can be integrated with Semantic Kernel and Azure OpenAI to answer questions from the database based on the schemas provided. 

A multi-shot approach is used for SQL generation for more reliable results and reduced token usage. More details can be found in the README.md.

In [1]:
import logging
import os
import yaml
import dotenv
import json
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
)
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.kernel import Kernel
from plugins.vector_based_sql_plugin.vector_based_sql_plugin import VectorBasedSQLPlugin
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig
from IPython.display import display, Markdown
from utils.sql import fetch_queries_from_cache, add_queries_to_cache
logging.basicConfig(level=logging.INFO)

## Kernel Setup

In [2]:
dotenv.load_dotenv()
kernel = Kernel()

## Set up GPT connections

In [3]:
service_id = "chat"

In [4]:
chat_service = AzureChatCompletion(
    service_id=service_id,
    deployment_name=os.environ["OpenAI__CompletionDeployment"],
    endpoint=os.environ["OpenAI__Endpoint"],
    api_key=os.environ["OpenAI__ApiKey"],
)
kernel.add_service(chat_service)

In [5]:
# Register the SQL Plugin with the Database name to use.
sql_plugin = VectorBasedSQLPlugin(database=os.environ["Text2Sql__DatabaseName"])
kernel.add_plugin(sql_plugin, "SQL")

KernelPlugin(name='SQL', description=None, functions={'GetEntitySchema': KernelFunctionFromMethod(metadata=KernelFunctionMetadata(name='GetEntitySchema', plugin_name='SQL', description='Gets the schema of a view or table in the SQL Database by selecting the most relevant entity based on the search term. Extract key terms from the user question and use these as the search term. Several entities may be returned.', parameters=[KernelParameterMetadata(name='text', description='The text to run a semantic search against. Relevant entities will be returned.', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={'type': 'string', 'description': 'The text to run a semantic search against. Relevant entities will be returned.'}, function_schema_include=True)], is_prompt=False, is_asynchronous=True, return_parameter=KernelParameterMetadata(name='return', description='', default_value=None, type_='str', is_required=True, type_object=<class 'str'>, schema_data={

## Prompt Setup

In [6]:
# Load prompt and execution settings from the file
with open("./query_cache_based_prompt.yaml", "r") as file:
    data = yaml.safe_load(file.read())
    prompt_template_config = PromptTemplateConfig(**data)

In [7]:
chat_function = kernel.add_function(
    prompt_template_config=prompt_template_config,
    plugin_name="ChatBot",
    function_name="Chat",
)

## ChatBot setup

In [8]:
history = ChatHistory()

In [9]:
async def ask_question(question: str, chat_history: ChatHistory) -> str:
    """Asks a question to the chatbot and returns the answer.
    
    Args:
        question (str): The question to ask the chatbot.
        chat_history (ChatHistory): The chat history object.
        
    Returns:
        str: The answer from the chatbot.
    """

    formatted_sql_cache_string = await fetch_queries_from_cache(question)

    # Create important information prompt that contains the SQL database information.
    engine_specific_rules = "Use TOP X to limit the number of rows returned instead of LIMIT X. NEVER USE LIMIT X as it produces a syntax error."
    important_information_prompt = f"""
    [SQL DATABASE INFORMATION]
    {sql_plugin.system_prompt(engine_specific_rules=engine_specific_rules)}
    [END SQL DATABASE INFORMATION]
    """

    question_string = f"{question}\n{formatted_sql_cache_string}"

    arguments = KernelArguments()
    arguments["chat_history"] = chat_history
    arguments["important_information"] = important_information_prompt
    arguments["user_input"] = question_string

    logging.info("Question: %s", question)

    answer = await kernel.invoke(
        function_name="Chat",
        plugin_name="ChatBot",
        arguments=arguments,
        chat_history=chat_history,
    )

    logging.info("Answer: %s", answer)

    # Log the question and answer to the chat history.
    chat_history.add_user_message(question)
    chat_history.add_message({"role": "assistant", "message": answer})

    json_answer = json.loads(str(answer))

    display(Markdown(json_answer["answer"]))

    await add_queries_to_cache(question, json_answer)

In [10]:
# Run the first time with no entries in the cache
await ask_question("What are the different product categories we have?", history)

INFO:httpx:HTTP Request: POST https://open-ai-gpt-001.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://open-ai-vector-db.search.windows.net/indexes('text-2-sql-query-cache-index')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '34853'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '1b6bc2fc-76a2-11ef-adb5-0242ac110002'
    'User-Agent': 'azsdk-python-search-documents/11.6.0b4 Python/3.12.6 (Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.36)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streami

The different product categories we have are:

- Bib-Shorts
- Bike Racks
- Bike Stands
- Bottles and Cages
- Bottom Brackets
- Brakes
- Caps
- Chains
- Cleaners
- Cranksets
- Derailleurs
- Fenders
- Forks
- Gloves
- Handlebars
- Headsets
- Helmets
- Hydration Packs
- Jerseys
- Lights
- Locks
- Mountain Bikes
- Mountain Frames
- Panniers
- Pedals
- Pumps
- Road Bikes
- Road Frames
- Saddles
- Shorts
- Socks
- Tights
- Tires and Tubes
- Touring Bikes
- Touring Frames
- Vests
- Wheels

INFO:root:Document: {'Question': 'What are the different product categories we have?', 'Query': 'SELECT DISTINCT ProductCategoryName FROM SalesLT.vGetAllCategories;', 'Schemas': [{'Entity': 'vGetAllCategories', 'Columns': [{'Name': 'ProductCategoryID', 'Definition': 'A unique identifier for each product category. This ID is used to reference specific categories.', 'Type': 'INT'}, {'Name': 'ParentProductCategoryName', 'Definition': 'The name of the parent product category. This represents the top-level category under which subcategories are grouped.', 'Type': 'NVARCHAR(50)'}, {'Name': 'ProductCategoryName', 'Definition': 'The name of the product category. This can refer to either a top-level category or a subcategory, depending on the context.', 'Type': 'NVARCHAR(50)'}]}]}
INFO:root:Vector Fields: {'Question': 'QuestionEmbedding'}
INFO:root:Vector Fields: {'Question': 'QuestionEmbedding'}
INFO:httpx:HTTP Request: POST https://open-ai-gpt-001.openai.azure.com/openai/deployments/text-embe

In [11]:
# run the second time when the question is cached
await ask_question("What are the different product categories we have?", history)

INFO:httpx:HTTP Request: POST https://open-ai-gpt-001.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://open-ai-vector-db.search.windows.net/indexes('text-2-sql-query-cache-index')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '34853'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '20648a0a-76a2-11ef-adb5-0242ac110002'
    'User-Agent': 'azsdk-python-search-documents/11.6.0b4 Python/3.12.6 (Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.36)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streami

The different product categories we have are as follows:

- Bib-Shorts
- Bike Racks
- Bike Stands
- Bottles and Cages
- Bottom Brackets
- Brakes
- Caps
- Chains
- Cleaners
- Cranksets
- Derailleurs
- Fenders
- Forks
- Gloves
- Handlebars
- Headsets
- Helmets
- Hydration Packs
- Jerseys
- Lights
- Locks
- Mountain Bikes
- Mountain Frames
- Panniers
- Pedals
- Pumps
- Road Bikes
- Road Frames
- Saddles
- Shorts
- Socks
- Tights
- Tires and Tubes
- Touring Bikes
- Touring Frames
- Vests
- Wheels

INFO:root:Document: {'Question': 'What are the different product categories we have?', 'Query': 'SELECT DISTINCT ProductCategoryName FROM SalesLT.vGetAllCategories;', 'Schemas': [{'Entity': 'vGetAllCategories', 'Columns': [{'Name': 'ProductCategoryID', 'Definition': 'A unique identifier for each product category. This ID is used to reference specific categories.', 'Type': 'INT'}, {'Name': 'ParentProductCategoryName', 'Definition': 'The name of the parent product category. This represents the top-level category under which subcategories are grouped.', 'Type': 'NVARCHAR(50)'}, {'Name': 'ProductCategoryName', 'Definition': 'The name of the product category. This can refer to either a top-level category or a subcategory, depending on the context.', 'Type': 'NVARCHAR(50)'}]}]}
INFO:root:Vector Fields: {'Question': 'QuestionEmbedding'}
INFO:root:Vector Fields: {'Question': 'QuestionEmbedding'}
INFO:httpx:HTTP Request: POST https://open-ai-gpt-001.openai.azure.com/openai/deployments/text-embe

In [12]:
# Run again with a different question
await ask_question("How many different product categories do we have?", history)

INFO:httpx:HTTP Request: POST https://open-ai-gpt-001.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version=2023-03-15-preview "HTTP/1.1 200 OK"
INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://open-ai-vector-db.search.windows.net/indexes('text-2-sql-query-cache-index')/docs/search.post.search?api-version=REDACTED'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '34720'
    'api-key': 'REDACTED'
    'Accept': 'application/json;odata.metadata=none'
    'x-ms-client-request-id': '26769816-76a2-11ef-adb5-0242ac110002'
    'User-Agent': 'azsdk-python-search-documents/11.6.0b4 Python/3.12.6 (Linux-5.15.153.1-microsoft-standard-WSL2-x86_64-with-glibc2.36)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; odata.metadata=none; odata.streami

We have a total of 37 different product categories in our database [1].

INFO:root:Document: {'Question': 'How many different product categories do we have?', 'Query': 'SELECT COUNT(DISTINCT ProductCategoryName) AS CategoryCount FROM SalesLT.vGetAllCategories;', 'Schemas': [{'Entity': 'vGetAllCategories', 'Columns': [{'Name': 'ProductCategoryID', 'Definition': 'A unique identifier for each product category. This ID is used to reference specific categories.', 'Type': 'INT'}, {'Name': 'ParentProductCategoryName', 'Definition': 'The name of the parent product category. This represents the top-level category under which subcategories are grouped.', 'Type': 'NVARCHAR(50)'}, {'Name': 'ProductCategoryName', 'Definition': 'The name of the product category. This can refer to either a top-level category or a subcategory, depending on the context.', 'Type': 'NVARCHAR(50)'}]}]}
INFO:root:Vector Fields: {'Question': 'QuestionEmbedding'}
INFO:httpx:HTTP Request: POST https://open-ai-gpt-001.openai.azure.com/openai/deployments/text-embedding-ada-002/embeddings?api-version